In [2]:
import tensorflow as tf
import pickle
import numpy as np
import timeit
from load_cifar import *
import time

In [3]:
tf.reset_default_graph()
validation_data, validation_labels = load_preprocessed_validation_batch()

<h1>Hyperparameter</h1>

In [4]:
lr = 0.0012
batch_size = 64
num_epochs = 10
num_classes = 10

<h1>Placeholder</h1>

In [5]:
X = tf.placeholder(tf.float32,[None,32,32,3],name='X')
Y = tf.placeholder(tf.int32,[None,num_classes],name='Y')
Training = tf.placeholder(tf.bool, None)

<h1>LeNet-5</h1>

In [6]:
bias_1 = tf.Variable(tf.zeros(6))
C1 = tf.layers.conv2d(X, filters=6, kernel_size=5, activation=tf.nn.relu) + bias_1

S2 = tf.layers.max_pooling2d(C1, pool_size=2, strides=2)

S2_bn = tf.layers.batch_normalization(S2)

bias_2 = tf.Variable(tf.zeros(16))
C3 = tf.layers.conv2d(S2_bn, filters=16, kernel_size=5, activation=tf.nn.relu) + bias_2

S4 = tf.layers.average_pooling2d(C3, pool_size=2, strides=2)

S4_bn = tf.layers.batch_normalization(S4)

bias_3 = tf.Variable(tf.zeros(120))
F5 = tf.layers.dense(tf.layers.flatten(S4_bn), units=120, activation=tf.nn.relu) + bias_3

F5 = tf.layers.dropout(F5, rate=0.12, training=Training)

# F5_bn = tf.layers.batch_normalization(F5)

bias_4 = tf.Variable(tf.zeros(84))
F6 = tf.layers.dense(F5, units=84, activation=tf.nn.relu) + bias_4

F6 = tf.layers.dropout(F6, rate=0.23, training=Training)

# F6_bn = tf.layers.batch_normalization(F6)

bias_5 = tf.Variable(tf.zeros(10))
logits = tf.layers.dense(F6, units=10) + bias_5

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Use keras.layers.max_pooling2d instead.
Instructions for updating:
Use keras.layers.batch_normalization instead.
Instructions for updating:
Use keras.layers.average_pooling2d instead.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


<h1>Cost and Optimization</h1>

In [7]:
# define loss
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,labels=Y),name='loss')
l2_loss = tf.losses.get_regularization_loss()
loss = loss + l2_loss
# define optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=lr)
train_op = optimizer.minimize(loss)

# compare the predicted labels with true labels
correct_pred = tf.equal(tf.argmax(tf.nn.softmax(logits),1),tf.argmax(Y,1))

# compute the accuracy by taking average
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32),name='accuracy')

<h1>Training, validation and testing</h1>
<h2>Train your model only 10 epochs.</h2>
<h2>1.Print out validation accuracy after each training epoch</h2>
<h2>2.Print out training time for each training epoch</h2>
<h2>3.Print out testing accuracy</h2>

In [8]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for i in range(num_epochs):
        s = time.time()
        batch_id = 1
        while batch_id < 6:
            training_data = load_preprocessed_training_batch(batch_id, batch_size)
            for data, labels in training_data:
                sess.run(train_op, feed_dict = {X: np.reshape(data, (-1, 32, 32, 3)), Y: labels, Training: True})
            batch_id = batch_id + 1
        val_acc = sess.run(accuracy, feed_dict={X: np.reshape(validation_data, (-1, 32, 32, 3)), Y: validation_labels, Training: False})
        f = time.time()
        print('Epoch:' + str(i) + '   Time:{:.3f}'.format(f - s) + '   Current validation accuracy: ' + str(val_acc))
    test_data, test_labels = load_all_test_batch()
    test_acc = sess.run(accuracy, feed_dict = {X: np.reshape(test_data, (-1, 32, 32, 3)), Y: test_labels, Training: False})
    print('Test accuracy: ')
    print(test_acc)

Epoch:0   Time:7.673   Current validation accuracy: 0.4332
Epoch:1   Time:4.045   Current validation accuracy: 0.4984
Epoch:2   Time:4.073   Current validation accuracy: 0.5296
Epoch:3   Time:4.110   Current validation accuracy: 0.5446
Epoch:4   Time:4.094   Current validation accuracy: 0.5504
Epoch:5   Time:4.105   Current validation accuracy: 0.5614
Epoch:6   Time:4.107   Current validation accuracy: 0.5676
Epoch:7   Time:4.084   Current validation accuracy: 0.5742
Epoch:8   Time:4.117   Current validation accuracy: 0.5712
Epoch:9   Time:4.157   Current validation accuracy: 0.5752
Test accuracy: 
0.5724
